In [178]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [ ]:
page = requests.get('https://www.larvalabs.com/cryptopunks/topsales')
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify)

In [180]:
punk_id = []
for ids in soup.find_all('div', attrs={'class': 'm-t-10'}):
  punk_id.append([ids.get_text()])

df_id = pd.DataFrame(punk_id)

In [181]:
price = []
for prices in soup.find_all('div', attrs = {'class': 'col-md-2 col-sm-3 col-xs-6 container-punk-event-large'}):
  price.append(prices.find_all('div')[4].get_text())

df_price = pd.DataFrame(price)

In [182]:
date = []
for dates in soup.find_all('div', attrs = {'class': 'col-md-2 col-sm-3 col-xs-6 container-punk-event-large'}):
  date.append(dates.find_all('div')[5].get_text())

df_date = pd.DataFrame(date)

In [183]:
df_top_sales = df_id
df_top_sales = df_top_sales.rename(columns={0:'punk_id'})
df_top_sales['punk_id']= df_top_sales['punk_id'].replace({'#':''}, regex=True)
df_top_sales['punk_id'] = df_top_sales['punk_id'].astype(str).astype(int)
df_top_sales.head()

,punk_id
0,7804
1,3100
2,6965
3,2140
4,3011


In [184]:
#Join Prices
df_top_sales = df_top_sales.join(df_price, on=df_top_sales.index, how='left', lsuffix='_', rsuffix='_')
df_top_sales = df_top_sales.rename(columns={0:'prices'})

#Split Prices into two columns
df_top_sales[['price_in_eth', 'price_in_dollars']] = df_top_sales['prices'].str.split(' ', 1, expand=True)
del df_top_sales['prices']
df_top_sales.head()

,punk_id,price_in_eth,price_in_dollars
0,7804,4.2KΞ,($7.57M)
1,3100,4.2KΞ,($7.58M)
2,6965,800Ξ,($1.54M)
3,2140,750Ξ,($1.18M)
4,3011,667Ξ,($1.76M)


In [185]:
#Remove special characters
df_top_sales['price_in_eth']= df_top_sales['price_in_eth'].replace({'Ξ':''}, regex=True)
df_top_sales['price_in_dollars'] = df_top_sales['price_in_dollars'].replace(to_replace='\(', value="", regex=True)
df_top_sales['price_in_dollars'] = df_top_sales['price_in_dollars'].replace(to_replace='\)', value="", regex=True)
df_top_sales['price_in_dollars'] = df_top_sales['price_in_dollars'].str.replace('$', '')
df_top_sales.head()

,punk_id,price_in_eth,price_in_dollars
0,7804,4.2K,7.57M
1,3100,4.2K,7.58M
2,6965,800,1.54M
3,2140,750,1.18M
4,3011,667,1.76M


In [186]:
#Convert strings into numbers
df_top_sales['price_in_eth'] = df_top_sales['price_in_eth'].replace({'K': '*1e3'}, regex=True).map(pd.eval).astype(int)
df_top_sales['price_in_dollars'] = df_top_sales['price_in_dollars'].replace({'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(int)
df_top_sales.head()

,punk_id,price_in_eth,price_in_dollars
0,7804,4200,7570000
1,3100,4200,7580000
2,6965,800,1540000
3,2140,750,1180000
4,3011,667,1760000


In [187]:
#Join Dates
df_top_sales = df_top_sales.join(df_date, on=df_top_sales.index, how='left', lsuffix='_', rsuffix='_')
df_top_sales = df_top_sales.rename(columns={0:'date_of_sale'})
df_top_sales['date_of_sale'] = pd.to_datetime(df_top_sales['date_of_sale'], infer_datetime_format=True)

In [189]:
df_top_sales

,punk_id,price_in_eth,price_in_dollars,date_of_sale
0,7804,4200,7570000,2021-03-11
1,3100,4200,7580000,2021-03-11
2,6965,800,1540000,2021-02-19
3,2140,750,1180000,2021-03-02
4,3011,667,1760000,2021-04-27
5,4156,650,1250000,2021-02-18
6,2890,605,761890,2021-01-23
7,6487,550,1050000,2021-02-22
8,3393,500,954650,2021-03-13
9,2484,480,1080000,2021-04-19
